# **<font color='greee'>Project  </font>** **<font color='pink'> Alzheimer’s MRI images – Extracting numerical and statistical features for classification  </font>**

<font color='orange'>Professor: </font> Osnat Bar-Shira

<font color='orange'>TA: </font> Benjamin Menashe

<font color='orange'>Students: </font>

Aracely Gutiérrez Lomelí G37064824

Tali Rozenson 208160937

Kanykei Mairambekova AC3188924

<font color='pink'>Note: </font>

The analysis of results is done in the report to avoid repeating descriptions both in the document and in the coding notebook.



## <font color='orange'>0. Imports</font>

In [3]:
# For creating training and test drive folders
import os
import shutil
import random

# Using Google drive as storage
from google.colab import drive
drive.mount('/content/drive')

# Pytorch for neural networks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# For time recording
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## <font color='orange'>1. First trial: Downsampling</font>

Drive folders for train and test set in a ratio of 80% and 20%, respectively.

Classes are imbalanced. It was decided to downsample every label into the minimum amount of images in any of the classes, 64 images for the moderate demented label. Therefore, the first trial only included 64 images per label: 51 for training and 13 for testing.

#### <font color='cyan'>1.1 Creating train drive folders for each label</font>

In [2]:
# Define the paths to folders
mild_path = '/content/drive/MyDrive/Dataset/Mild_Demented'
moderate_path = '/content/drive/MyDrive/Dataset/Moderate_Demented'
non_path = '/content/drive/MyDrive/Dataset/Non_Demented'
very_mild_path = '/content/drive/MyDrive/Dataset/Very_Mild_Demented'

# Function to randomly select and copy 51 images from a folder
def select_random_images(source_path, dest_path, num_images=51):
    file_list = os.listdir(source_path)
    selected_files = random.sample(file_list, min(num_images, len(file_list)))

    # Create the destination folder if it doesn't exist
    os.makedirs(dest_path, exist_ok=True)

    # Copy selected files to the destination folder
    for file_name in selected_files:
        source_file_path = os.path.join(source_path, file_name)
        dest_file_path = os.path.join(dest_path, file_name)
        shutil.copy2(source_file_path, dest_file_path)

# Randomly select and copy 51 images from each folder
select_random_images(mild_path, '/content/drive/MyDrive/Dataset/Train/Mild')
select_random_images(moderate_path, '/content/drive/MyDrive/Dataset/Train/Moderate')
select_random_images(non_path, '/content/drive/MyDrive/Dataset/Train/Non')
select_random_images(very_mild_path, '/content/drive/MyDrive/Dataset/Train/Very_Mild')

#### <font color='cyan'>1.2 Creating test drive folders for each label</font>

In [3]:
# Define the paths to folders
mild_path = '/content/drive/MyDrive/Dataset/Mild_Demented'
moderate_path = '/content/drive/MyDrive/Dataset/Moderate_Demented'
non_path = '/content/drive/MyDrive/Dataset/Non_Demented'
very_mild_path = '/content/drive/MyDrive/Dataset/Very_Mild_Demented'

# Function to randomly select and copy 13 images from a folder
def select_random_images(source_path, dest_path, num_images=13):
    file_list = os.listdir(source_path)
    selected_files = random.sample(file_list, min(num_images, len(file_list)))

    # Create the destination folder if it doesn't exist
    os.makedirs(dest_path, exist_ok=True)

    # Copy selected files to the destination folder
    for file_name in selected_files:
        source_file_path = os.path.join(source_path, file_name)
        dest_file_path = os.path.join(dest_path, file_name)
        shutil.copy2(source_file_path, dest_file_path)

# Randomly select and copy 13 images from each folder
select_random_images(mild_path, '/content/drive/MyDrive/Dataset/Test/Mild')
select_random_images(moderate_path, '/content/drive/MyDrive/Dataset/Test/Moderate')
select_random_images(non_path, '/content/drive/MyDrive/Dataset/Test/Non')
select_random_images(very_mild_path, '/content/drive/MyDrive/Dataset/Test/Very_Mild')

#### <font color='cyan'>1.3 Transforming images to tensors and normalizing</font>

Each pixel is usually scaled to range of 0 to 1. Then, in the normalization step, we would usually prefer a mean of 0 and standard deviation of 1 but another common normalization practice is0.5 as mean and 0.5 as standard deviation.

In [5]:
# Define the transformation for your images
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))
])

#### <font color='cyan'>1.4 Defining the labels for the images in train and test sets</font>

In the previous cells, we have divided our images by folders containing the name of the label of the pictures. In the next two cells, we define the label of each image as the name of the folder it is contained in.


The datasets.ImageFolder class automatically assigns labels based on
the subfolder names inside the specified root folder.

The batch size is calculates as *Effective batch size = Total samples / Update steps* and since we have very few samples in the test set, we might use 6 update steps and a batch size of 2.

In [19]:
# Define the path to the "Train" folder
train_root = '/content/drive/MyDrive/Dataset/Train'

# Create the ImageFolder dataset
train_dataset = datasets.ImageFolder(train_root, transform=transform)

# You can use DataLoader to create batches for training
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)

In [20]:
# Define the path to the "Test" folder
test_root = '/content/drive/MyDrive/Dataset/Test'

# Create the ImageFolder dataset
test_dataset = datasets.ImageFolder(test_root, transform=transform)

# You can use DataLoader to create batches for training
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=True)

#### <font color='cyan'>1.5 CNN Model</font>

A convolutional neural network was used as the classification model.

Although the images are supposed to be grayscale (one input channel), the algorithm detected three as in RGB images, so the CNN model algorithm includes three input channels. Importantly, the number of output channels is just a design choice in the architecture of the model, and the kernel size means the kernel is squared 5x5. The output size for the fully connected layers corresponds to the number of classes (multiclass classification task: 4 labels = mild, moderate, very mild and non-demented).

In [21]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5) # nn.Conv2d = Convolutional layers for 2D and 1D data
        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # Fully Connected Layers
        self.fc1 = nn.Linear(16 * 29 * 29, 120)  # 16 channels, 29x29 spatial dimensions = Input to the fully connected layers
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)  # Output size corresponds to the number of classes (4 = Mild, moderate, very mild demented and nondemented)

    def forward(self, x):
        # First Convolutional Layer with ReLU activation and max pooling
        x = F.relu(self.conv1(x)) # size will be reduced by N-F/stride +1 = 128-5/1 + 1 = 124
        x = F.max_pool2d(x, 2) #  124/2 = 62
        # Second Convolutional Layer with ReLU activation and max pooling
        x = F.relu(self.conv2(x)) # 62-5/1 + 1 = 58
        x = F.max_pool2d(x, 2) # 58/2 = 29
        # Flatten before passing through fully connected layers
        x = x.view(-1, self.num_flat_features(x))
        # Fully Connected Layers with ReLU activation
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Output layer (no activation as it's handled by CrossEntropyLoss)
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # Exclude batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [22]:
# Instantiate the model
cnn_model = SimpleCNN()

#### <font color='cyan'>1.6 Loss function and optimizer</font>

Cross entropy was applied to this task due to the recommended use of multiclass classification. This function is combined with softmax which gives a probability distribution of the classes while the cross entropy function is the loss function measuring how these predicted probabilities guess the true answer.

In [23]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(cnn_model.parameters(),lr=0.001)

#### <font color='cyan'>1.7 Implementation of the model</font>
By testing and analyzing the number or epochs in previous attempts, it was found that 10 epochs is enough to converge.

In [24]:
n_epochs = 10
t_start = time.time()

for epoch in range(n_epochs):
    correct_train = 0
    total_train = 0

    for batch_idx, batch in enumerate(train_dataloader):
        samples, labels = batch

        # Forward pass
        y_pred = cnn_model(samples)
        loss = criterion(y_pred, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Zero the gradients after updating
        optimizer.zero_grad()

        # Calculate training accuracy
        _, predicted = torch.max(y_pred.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Print training accuracy after each epoch
    train_accuracy = correct_train / total_train
    print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy * 100:.2f}%')

t_end = time.time()
print('Total training time:', t_end - t_start)

Epoch 1/10, Loss: 1.3858, Training Accuracy: 24.02%
Epoch 2/10, Loss: 1.3455, Training Accuracy: 34.80%
Epoch 3/10, Loss: 1.3016, Training Accuracy: 46.08%
Epoch 4/10, Loss: 0.2853, Training Accuracy: 67.65%
Epoch 5/10, Loss: 0.4892, Training Accuracy: 80.88%
Epoch 6/10, Loss: 1.1534, Training Accuracy: 88.73%
Epoch 7/10, Loss: 0.0368, Training Accuracy: 94.61%
Epoch 8/10, Loss: 0.0153, Training Accuracy: 99.02%
Epoch 9/10, Loss: 0.0004, Training Accuracy: 100.00%
Epoch 10/10, Loss: 0.0000, Training Accuracy: 100.00%
Total training time: 63.22462725639343


## <font color='orange'>2. Second trial: Changing the loss function - Focal loss</font>

Focal loss was used because it is said to encourage the model to learn on misclassified samples, and reduce the impact of the overrepresentation of a certain class (Niyaz, 2023)

In [32]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.alpha is not None:
            focal_loss = self.alpha * focal_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [33]:
criterion_focal = FocalLoss(gamma=2, alpha=None, reduction='mean')
optimizer=torch.optim.Adam(cnn_model.parameters(),lr=0.001)

In [34]:
n_epochs = 10
t_start = time.time()

for epoch in range(n_epochs):
    correct_train = 0
    total_train = 0

    for batch_idx, batch in enumerate(train_dataloader):
        samples, labels = batch

        # Forward pass
        y_pred = cnn_model(samples)
        loss = criterion_focal(y_pred, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Zero the gradients after updating
        optimizer.zero_grad()

        # Calculate training accuracy
        _, predicted = torch.max(y_pred.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Print training accuracy after each epoch
    train_accuracy = correct_train / total_train
    print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy * 100:.2f}%')

t_end = time.time()
print('Total training time:', t_end - t_start)


Epoch 1/10, Loss: 0.3716, Training Accuracy: 55.88%
Epoch 2/10, Loss: 0.0311, Training Accuracy: 66.67%
Epoch 3/10, Loss: 0.7939, Training Accuracy: 77.45%
Epoch 4/10, Loss: 0.1614, Training Accuracy: 82.35%
Epoch 5/10, Loss: 0.0276, Training Accuracy: 86.27%
Epoch 6/10, Loss: 0.0005, Training Accuracy: 94.12%
Epoch 7/10, Loss: 0.0031, Training Accuracy: 98.53%
Epoch 8/10, Loss: 0.0000, Training Accuracy: 100.00%
Epoch 9/10, Loss: 0.0096, Training Accuracy: 100.00%
Epoch 10/10, Loss: 0.0050, Training Accuracy: 100.00%
Total training time: 56.176963567733765


## <font color='orange'>3. Third trial: Varying parameters in CNN model  - Drop out</font>
Dropout was used to prevent overfitting

In [35]:
import torch.nn as nn
import torch.nn.functional as F

class CNNWithDropout(nn.Module):
    def __init__(self):
        super(CNNWithDropout, self).__init__()
        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(16 * 29 * 29, 120)
        self.dropout1 = nn.Dropout(0.5)  # Adjust dropout probability as needed
        self.fc2 = nn.Linear(120, 84)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # Exclude batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [36]:
# Instantiate the model
cnn_dropout_model = CNNWithDropout()

In [37]:
n_epochs = 10
t_start = time.time()

for epoch in range(n_epochs):
    correct_train = 0
    total_train = 0

    for batch_idx, batch in enumerate(train_dataloader):
        samples, labels = batch

        # Forward pass
        y_pred = cnn_dropout_model(samples)
        loss = criterion_focal(y_pred, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Zero the gradients after updating
        optimizer.zero_grad()

        # Calculate training accuracy
        _, predicted = torch.max(y_pred.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Print training accuracy after each epoch
    train_accuracy = correct_train / total_train
    print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy * 100:.2f}%')

t_end = time.time()
print('Total training time:', t_end - t_start)

Epoch 1/10, Loss: 0.8238, Training Accuracy: 25.49%
Epoch 2/10, Loss: 0.7807, Training Accuracy: 26.47%
Epoch 3/10, Loss: 0.7459, Training Accuracy: 26.47%
Epoch 4/10, Loss: 0.7544, Training Accuracy: 23.04%
Epoch 5/10, Loss: 0.8679, Training Accuracy: 26.47%
Epoch 6/10, Loss: 0.7739, Training Accuracy: 23.53%
Epoch 7/10, Loss: 0.7658, Training Accuracy: 22.55%
Epoch 8/10, Loss: 0.8895, Training Accuracy: 29.90%
Epoch 9/10, Loss: 0.8216, Training Accuracy: 20.59%
Epoch 10/10, Loss: 0.8631, Training Accuracy: 22.55%
Total training time: 26.403857231140137


## <font color='orange'>4. Fourth trial: Using the entire dataset</font>

Analyzing the possibility to classify correctly with the entire dataset with focal loss, considering class imbalance.
#### <font color='cyan'>4.1 Creating train drive folders for each label</font>

In [98]:
# Define the paths to your folders
mild_path = '/content/drive/MyDrive/Dataset/Mild_Demented'
moderate_path = '/content/drive/MyDrive/Dataset/Moderate_Demented'
non_path = '/content/drive/MyDrive/Dataset/Non_Demented'
very_mild_path = '/content/drive/MyDrive/Dataset/Very_Mild_Demented'

# Function to randomly select and copy 80% of the images from a folder
def select_random_images(source_path, dest_path, percentage=0.8):
    file_list = os.listdir(source_path)
    num_images = int(len(file_list) * percentage)
    selected_files = random.sample(file_list, min(num_images, len(file_list)))

    # Create the destination folder if it doesn't exist
    os.makedirs(dest_path, exist_ok=True)

    # Copy selected files to the destination folder
    for file_name in selected_files:
        source_file_path = os.path.join(source_path, file_name)
        dest_file_path = os.path.join(dest_path, file_name)
        shutil.copy2(source_file_path, dest_file_path)

# Randomly select and copy 80% of the images from each folder
select_random_images(mild_path, '/content/drive/MyDrive/Dataset/Large_Train/Mild')
select_random_images(moderate_path, '/content/drive/MyDrive/Dataset/Large_Train/Moderate')
select_random_images(non_path, '/content/drive/MyDrive/Dataset/Large_Train/Non')
select_random_images(very_mild_path, '/content/drive/MyDrive/Dataset/Large_Train/Very_Mild')

#### <font color='cyan'>4.2 Creating test drive folders for each label</font>

In [99]:
# Define the paths to your folders
mild_path = '/content/drive/MyDrive/Dataset/Mild_Demented'
moderate_path = '/content/drive/MyDrive/Dataset/Moderate_Demented'
non_path = '/content/drive/MyDrive/Dataset/Non_Demented'
very_mild_path = '/content/drive/MyDrive/Dataset/Very_Mild_Demented'

# Function to randomly select and copy 20% of the images from a folder
def select_random_images(source_path, dest_path, percentage=0.2):
    file_list = os.listdir(source_path)
    num_images = int(len(file_list) * percentage)
    selected_files = random.sample(file_list, min(num_images, len(file_list)))

    # Create the destination folder if it doesn't exist
    os.makedirs(dest_path, exist_ok=True)

    # Copy selected files to the destination folder
    for file_name in selected_files:
        source_file_path = os.path.join(source_path, file_name)
        dest_file_path = os.path.join(dest_path, file_name)
        shutil.copy2(source_file_path, dest_file_path)

# Randomly select and copy 20% of the images from each folder
select_random_images(mild_path, '/content/drive/MyDrive/Dataset/Large_Test/Mild')
select_random_images(moderate_path, '/content/drive/MyDrive/Dataset/Large_Test/Moderate')
select_random_images(non_path, '/content/drive/MyDrive/Dataset/Large_Test/Non')
select_random_images(very_mild_path, '/content/drive/MyDrive/Dataset/Large_Test/Very_Mild')

#### <font color='cyan'>4.3 Defining the labels for the images in train and test sets</font>

In [6]:
# Define the path to the "Train" folder
train_root = '/content/drive/MyDrive/Dataset/Large_Train'

# Create the ImageFolder dataset
train_dataset = datasets.ImageFolder(train_root, transform=transform)

# You can use DataLoader to create batches for training
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True) #Batch sizes= 6400*0.8= 5120 samples/

In [7]:
# Define the path to the "Test" folder
test_root = '/content/drive/MyDrive/Dataset/Large_Test'

# Create the ImageFolder dataset
test_dataset = datasets.ImageFolder(test_root, transform=transform)

# You can use DataLoader to create batches for training
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True) #Batch sizes= 6400*0.2= 1280 samples/

#### <font color='cyan'>4.4 Defining CNN model</font>

In [8]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5) # nn.Conv2d = Convolutional layers for 2D and 1D data
        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # Fully Connected Layers
        self.fc1 = nn.Linear(16 * 29 * 29, 120)  # 16 channels, 4x4 spatial dimensions
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)  # Output size corresponds to the number of classes (4 = Mild, moderate, very mild demented and nondemented)

    def forward(self, x):
        # First Convolutional Layer with ReLU activation and max pooling
        x = F.relu(self.conv1(x)) # size will be reduced by N-F/stride +1 = 128-5/1 + 1 = 124
        x = F.max_pool2d(x, 2) #  124/2 = 62
        # Second Convolutional Layer with ReLU activation and max pooling
        x = F.relu(self.conv2(x)) # 62-5/1 + 1 = 58
        x = F.max_pool2d(x, 2) # 58/2 = 29
        # Flatten before passing through fully connected layers
        x = x.view(-1, self.num_flat_features(x))
        # Fully Connected Layers with ReLU activation
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Output layer (no activation as it's handled by CrossEntropyLoss)
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # Exclude batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

#### <font color='cyan'>4.5 Instantiating the model</font>

In [9]:
# Instantiate the model
cnn_model = SimpleCNN()

#### <font color='cyan'>4.6 Defining focal loss</font>

In [10]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.alpha is not None:
            focal_loss = self.alpha * focal_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [11]:
criterion_focal = FocalLoss(gamma=2, alpha=None, reduction='mean')
optimizer=torch.optim.Adam(cnn_model.parameters(),lr=0.001)

#### <font color='cyan'>4.7 Implementing the model</font>

In [12]:
n_epochs = 20
t_start = time.time()

for epoch in range(n_epochs):
    correct_train = 0
    total_train = 0

    for batch_idx, batch in enumerate(train_dataloader):
        samples, labels = batch

        # Forward pass
        y_pred = cnn_model(samples)
        loss = criterion_focal(y_pred, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Zero the gradients after updating
        optimizer.zero_grad()

        # Calculate training accuracy
        _, predicted = torch.max(y_pred.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Print training accuracy after each epoch
    train_accuracy = correct_train / total_train
    print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy * 100:.2f}%')

t_end = time.time()
print('Total training time:', t_end - t_start)


Epoch 1/20, Loss: 0.3203, Training Accuracy: 53.17%
Epoch 2/20, Loss: 0.3308, Training Accuracy: 66.01%
Epoch 3/20, Loss: 0.1374, Training Accuracy: 80.62%
Epoch 4/20, Loss: 0.0571, Training Accuracy: 91.70%
Epoch 5/20, Loss: 0.0156, Training Accuracy: 96.21%
Epoch 6/20, Loss: 0.0077, Training Accuracy: 98.96%
Epoch 7/20, Loss: 0.0055, Training Accuracy: 99.49%
Epoch 8/20, Loss: 0.0093, Training Accuracy: 99.84%
Epoch 9/20, Loss: 0.0009, Training Accuracy: 99.96%
Epoch 10/20, Loss: 0.0002, Training Accuracy: 100.00%
Epoch 11/20, Loss: 0.0002, Training Accuracy: 100.00%
Epoch 12/20, Loss: 0.0001, Training Accuracy: 100.00%
Epoch 13/20, Loss: 0.0001, Training Accuracy: 100.00%
Epoch 14/20, Loss: 0.0001, Training Accuracy: 100.00%
Epoch 15/20, Loss: 0.0000, Training Accuracy: 100.00%
Epoch 16/20, Loss: 0.0000, Training Accuracy: 100.00%
Epoch 17/20, Loss: 0.0000, Training Accuracy: 100.00%
Epoch 18/20, Loss: 0.0000, Training Accuracy: 100.00%
Epoch 19/20, Loss: 0.0000, Training Accuracy: 

#### <font color='cyan'>4.8 Evaluating on test set</font>

In [13]:
# Initialize variables for accuracy calculation on the test set
correct_test = 0
total_test = 0

# Set the model to evaluation mode
cnn_model.eval()

# Iterate over the test set
with torch.no_grad():
    for batch_idx, batch in enumerate(test_dataloader):
        samples, labels = batch

        # Forward pass
        y_pred = cnn_model(samples)

        # Calculate test accuracy
        _, predicted = torch.max(y_pred.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

# Print test accuracy
test_accuracy = correct_test / total_test
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 99.53%
